[View in Colaboratory](https://colab.research.google.com/github/ruxandraburtica/recommender-systems/blob/master/Copy_of_3_content_based.ipynb)


# 3. Content-based recommender systems




## Imports

In [0]:
from io import BytesIO
import os
from ast import literal_eval

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import seaborn as sns



%matplotlib inline

## Read the data

In [0]:
def download_file(url, file_name):
    # First remove the archive_name file if it exists.
    if os.path.exists(file_name):
        os.remove(file_name)
    request = requests.get(url)
    with open(file_name, "wb") as fd:
        fd.write(request.content)


download_file('https://s3-eu-west-1.amazonaws.com/machine-learning-public/workshop/movies_metadata.csv', 'movies_metadata.csv')
download_file('https://s3-eu-west-1.amazonaws.com/machine-learning-public/workshop/keywords.csv', 'keywords.csv')
download_file('https://s3-eu-west-1.amazonaws.com/machine-learning-public/workshop/credits.csv', 'credits.csv')
download_file('https://s3-eu-west-1.amazonaws.com/machine-learning-public/workshop/links_small.csv', 'links_small.csv')

In [0]:
md = pd.read_csv('movies_metadata.csv')
keywords = pd.read_csv('keywords.csv')
credits = pd.read_csv('credits.csv')
links_small = pd.read_csv('links_small.csv')

In [0]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [0]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()

m = vote_counts.quantile(0.95)

In [0]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [0]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

In [0]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [0]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [0]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [0]:
qualified.head(15)

In [0]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [0]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [0]:
build_chart('Romance').head(15)

In [0]:
md = md.drop([19730, 29503, 35587])

In [0]:
md['id'] = md['id'].astype('int')

In [0]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
smd = md[md['id'].isin(links_small)]
smd.shape

In [0]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [0]:
smd.head()

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [0]:
tfidf_matrix.shape

In [0]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [0]:
md.shape

In [0]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [0]:
smd = md[md['id'].isin(links_small)]
smd.shape

In [0]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [0]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [0]:
smd['director'] = smd['crew'].apply(get_director)

In [0]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [0]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [0]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [0]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

In [0]:
s = s[s > 1]

In [0]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [0]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [0]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [0]:
smd['soup'] = smd['keywords'] +smd['cast']  + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [0]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [0]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [0]:
smd.head()

In [0]:
def get_recommendations(title):
    # Get movie index
    idx = indices[title]
    
    # Use cosine similarity function (cosine_sim) to get similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Order similarity descending and return the first X (let's say 50.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    
    # Extract movie indices for the obtained scores
    movie_indices = [i[0] for i in sim_scores]
    
    # Return movie titles
    return titles.iloc[movie_indices]

In [0]:
get_recommendations('The Dark Knight').head(10)